In [7]:
import os
import pandas as pd

from Bio import SeqIO
from zipfile import ZipFile

In [2]:
file_highly_expressed = r'../samples/moor_s1_highly_expressed_transcript.tsv'

In [3]:
df = pd.read_csv(file_highly_expressed, delimiter='\t')

In [4]:
df.head(2)

,Target ID,Gene Name,Measure Names,Qval Color,Number of target,Measure Values
0,ENSMUST00000000090,Cox5a,Apical Bias,Red,598,0.254249
1,ENSMUST00000000312,Cdh1,Apical Bias,Red,598,1.540903


In [5]:
df_apical = df[df['Measure Values'] > 0]
df_basal =  df[df['Measure Values'] < 0]
f'df_apical: {len(df_apical)}, df_basal: {len(df_basal)}'

'df_apical: 406, df_basal: 192'

In [8]:
# Load cdna, cds sequence
cdna_file = os.path.join(os.path.abspath(r'../samples'), 's1_cdna.fasta')
cds_file = os.path.join(os.path.abspath(r'../samples'), 's1_cds.fasta')
cdna_zipfile = os.path.join(os.path.abspath(r'../samples'), 's1_cdna.zip')
cds_zipfile = os.path.join(os.path.abspath(r'../samples'), 's1_cds.zip')

with ZipFile(cdna_zipfile, 'r') as zip_ref:
    zip_ref.extract(os.path.basename(cdna_file),os.path.dirname(cdna_file))

with ZipFile(cds_zipfile, 'r') as zip_ref:
    zip_ref.extract(os.path.basename(cds_file),os.path.dirname(cds_file))

dic_cds = dict()
with open(cds_file, "r") as fin:
    for rec in SeqIO.parse(fin, "fasta"):
        dic_cds[rec.id] =rec

dic_cdna = dict()
with open(cdna_file, "r") as fin:
    for rec in SeqIO.parse(fin, "fasta"):
        dic_cdna[rec.id] =rec

f'CDS: {len(dic_cds)}, CDNA: {len(dic_cdna)}'

'CDS: 7784, CDNA: 9858'

In [37]:
lst_apical_high = df_apical[[x in dic_cds.keys() for x in df_apical['Target ID']]]['Target ID']
print(f'number of highly expressed apical gene with sequence data accessible: {len(lst_apical_high)}')

number of highly expressed apical gene with sequence data accessible: 359


In [38]:
lst_apical_high

0      ENSMUST00000000090
1      ENSMUST00000000312
4      ENSMUST00000001963
5      ENSMUST00000002091
6      ENSMUST00000002303
              ...        
575    ENSMUST00000201833
581    ENSMUST00000203652
586    ENSMUST00000204601
587    ENSMUST00000204807
595    ENSMUST00000208086
Name: Target ID, Length: 359, dtype: object

In [39]:
lst_basal_high = df_basal[[x in dic_cds.keys() for x in df_basal['Target ID']]]['Target ID']
print(f'number of highly expressed basal gene with sequence data accessible: {len(lst_basal_high)}')

number of highly expressed basal gene with sequence data accessible: 103


In [47]:
def extract_3utr(lst_target):
    mismatch = list()
    errfind = list()
    dic_extr = dict()
    for cdskey in lst_target:
        _cds = dic_cds.get(cdskey)
        _cdna = dic_cdna.get(cdskey)
        start_ix = _cdna.seq.find(_cds.seq)
        if start_ix < 0:
            mismatch.append((cdskey, start_ix))
        else:
            _utr3 = _cdna.seq[start_ix + len(_cds.seq):]
            # It should be Zero
            length_verified = len(_cdna.seq) - start_ix - len(_cds.seq) - len(_utr3)
            if length_verified != 0:
                errfind.append((cdskey, length_verified))
            else:
                dic_extr[cdskey] = {'cdna': _cdna, 'cds':_cds, 'utr3':_utr3}
    print(f'# mismatch: {len(mismatch)}, # errfind: {len(errfind)}, # extr: {len(dic_extr)}')
    
    return dic_extr, mismatch, errfind

In [50]:
dic_apical, mismatch, errfind = extract_3utr(lst_apical_high)
dic_basal, mismatch, errfind = extract_3utr(lst_basal_high)

# mismatch: 8, # errfind: 0, # extr: 351
# mismatch: 2, # errfind: 0, # extr: 101


[('ENSMUST00000124121', -1),
 ('ENSMUST00000135569', -1),
 ('ENSMUST00000144150', -1),
 ('ENSMUST00000144529', -1),
 ('ENSMUST00000173115', -1),
 ('ENSMUST00000191404', -1),
 ('ENSMUST00000201833', -1),
 ('ENSMUST00000208086', -1)]